In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from pathlib import Path

# dummy data to test out donor imputation with
from sklearn.datasets import load_iris
import random

# for nearest neighbour imputation
from sklearn.impute import KNNImputer
from sklearn.neighbors import NearestNeighbors, BallTree
from sklearn.preprocessing import MinMaxScaler

# for type hints
import typing
from typing import List, Set, Dict, Tuple, Union, Optional

# for warning messages
import warnings

In [2]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 150

Main functions to write:
- Hot deck
- KNN imputation
- Trend imputation

## Parse Imputation Specifications

We could define how the file containing the list of variables to impute, its imputation class variables, type of imputation to apply and restrictions on imputed value should be formatted.

## Input Format

Based on the files given by the Singapore team, we assume that:

- categorical variables (*besides variables that begin with GO*) are one-hot encoded already ex. Q116, Q121
    - is this necessary if we are using hot deck imputation instead of distance-based donor imputation methods?
- categorical variables that take on alphanumerical values have not been one-hot encoded ex. GO_Q130, Age
- Don't Know responses are either denoted as -1, 88 or 888 (see data dictionary), refused responses are denoted as 99
    - See Explanatory Notes section of the data dictionary for more information.

Constraint to incorporate:

- Minimum values (prefixed by `MinValue`)

Questions for clients:

- Confirm what values we want to impute for. Is it just blank entries, unknown values or should we also impute for don't know and refused values?
- Ask about post-edits: would some pose constraints/limits during the imputation step, or will constraints be applied after the imputation step?

## Donor imputation processing steps

1. Data preparation

   - Bucket values of **variables to be imputed**
   - Standardize missing values for imputation: there are several values that can denote values to impute. _Verify with clients if they want to impute for refused/don't know responses._
   - Having a standard format for specifying variables to impute - like a config file.
   - Make one copy of data prior to donor imputation, and add column for donor use counter.

2. For applicable variables only, do historical imputation

   - Use functions 
   
4. Find donor

   - Use function nearest_neighbour_imputation to get donor value and its index.
   - Exclude donors with over 5 uses.

# Donor Imputation

## Utility Functions

In [3]:
def data_preprocessing_store_donor_info(data: pd.DataFrame, var_to_impute: str, imputation_class_vars: List[str], knn_distance: bool = True) -> pd.DataFrame:

    """Function that pre-processes data set to include variables pertaining to imputation.

    Variables added include: duplicate column of the variable to impute with suffix '_post_imputation', 
    imputation flag, imputation type (historical, KNN), imputed value, donor user ID,
    donor use count, donor distance, and donor imputation class details. 
    
    Donor-specific variables only apply to KNN imputation.
    
    Default values for these variables are:
        - imputation flag: 0
        - imputation type: '' (blank string)
        - imputed value: NA
        - donor user ID: -1
        - donor use count: 0
        - donor distance: NA
        - donor imputation class details: NA
    These variables are prefixed by the name of the variable to be imputed.

    Parameters
    ----------

    data : pd.DataFrame
        Pandas DataFrame of input data.

    var_to_impute : str
        Name of variable to impute.

    imputation_class_vars : list of str
        Names of imputation class variables.

    knn_distance : boolean (default = True)
        Distance of donor.

    Returns
    -------

    data_prepped : pd.DataFrame
        Pandas DataFrame of input data with additional columns pertaining to imputation details.
    """

    data_prepped = data.copy()

    # Create duplicate of variable to impute
    var_to_impute_post_imputation_name = f'{var_to_impute}_post_imputation'
    data_prepped[var_to_impute_post_imputation_name] = data_prepped[var_to_impute]

    # Create imputation flag for variable to impute.
    imputation_flag_col_name = f'{var_to_impute}_imputation_flag'
    data_prepped[imputation_flag_col_name] = 0

    # Create column that denotes type of imputation (KNN, historical).
    imputation_type_col_name = f'{var_to_impute}_imputation_type'
    data_prepped[imputation_type_col_name] = ''

    # Create column that contains donated, imputed value.
    # If imputation not required or no donor found, then imputed_value is NaN.
    imputed_val_col_name = f'{var_to_impute}_imputed_value'
    data_prepped[imputed_val_col_name] = pd.NA

    # Create column that stores userid of donor, if applicable.
    # If imputation not required or no donor found, then userid = -1.
    donor_userid_col_name = f'{var_to_impute}_donor_userid'
    data_prepped[donor_userid_col_name] = -1
    
    # Create donor count variable.
    donor_count_col_name = f'{var_to_impute}_donor_count'
    data_prepped[donor_count_col_name] = 0

    # Create columns for donor characteristics: values of imputation class variables.
    for var in imputation_class_vars:
        donor_imputation_class_var = f'{var_to_impute}_donor_{var}'
        data_prepped[donor_imputation_class_var] = pd.NA

    # Create column for donor distance.
    # If donor not obtained via KNN, then distance will default to NaN.
    donor_dist_col_name = f'{var_to_impute}_donor_distance'
    data_prepped[donor_dist_col_name] = pd.NA

    return data_prepped

In [4]:
def continuous_to_range(data: pd.DataFrame, column: str, range_interval_index: pd.IntervalIndex, range_label_dict: Dict) -> pd.Series:
    
    """Function that converts continuous values to bucketed ranges.

    Parameters
    ----------

    data : pd.DataFrame
        Pandas DataFrame of input data.

    column : str
        Name of column to convert continuous values to bucketed ranges.

    range_interval_index : pd.IntervalIndex
        Pandas Interval Index type that defines the buckets.
        Note that the left bound is exclusive, right bound is inclusive.

    range_label_dict : dict
        Dictionary with Pandas Interval Indices as keys and corresponding integer labels as values.

    Returns
    -------

    continuous_to_bucketed_series: pd.Series
        Pandas Series of column with bucketed ranges.
    """
    
    # returns a pandas Series that converts continuous values to buckets
    
    continuous_to_bucketed_series = pd.cut(data[column], range_interval_index, labels = range(1, len(range_interval_index) + 1))
    continuous_to_bucketed_series = continuous_to_bucketed_series.map(range_label_dict)
    return continuous_to_bucketed_series

In [5]:
def range_to_random_continuous(data: pd.DataFrame, column: str, range_to_continuous_dict: Dict) -> pd.Series:
 
    """Function that converts bucketed ranges to a randomly selected value within the range.

    Parameters
    ----------

    data : pd.DataFrame
        Pandas DataFrame of input data.

    column : str
        Name of column to convert continuous values to bucketed ranges.

    range_to_continuous_dict : dict
        Dictionary with integer labels of buckets as keys and tuples of range .

    Returns
    -------

    bucketed_to_continuous_series: pd.Series
        Pandas Series of column with numbers randomly selected from bucketed range.
    """

    bucketed_to_continuous_series = pd.Series(pd.NA, index=data.index)
    
    for row_num, val in enumerate(data[column]):
        if not pd.isna(val):
            bucketed_to_continuous_series[row_num] = random.randint(*range_to_continuous_dict[val]) #.astype(float)
        else:
            bucketed_to_continuous_series[row_num] = pd.NA

    return bucketed_to_continuous_series

In [6]:
values_to_impute = [-1, 88]

In [7]:
# convert continuous values to buckets
monetary_buckets = [
    (-0.001, 20),
    (20, 40), 
    (40, 60), 
    (60, 80), 
    (80, 100), 
    (100, 200), 
    (200, 300), 
    (300, 400), 
    (400, 500), 
    (500, 600), 
    (600, 700), 
    (700, 800), 
    (800, 900), 
    (900, 1000), 
    (1000, 1500), 
    (1500, 2000), 
    (2000, 2500), 
    (2500, 3000), 
    (3000, 3500), 
    (3500, 4000), 
    (4000, 4500), 
    (4500, 5000), 
    (5000, 5500), 
    (5500, 6000), 
    (6000, 6500), 
    (6500, 7000), 
    (7000, 7500), 
    (7500, 8000), 
    (8000, 8500), 
    (8500, 9000), 
    (9000, 9500), 
    (9500, 10000), 
    (10000, 15000), 
    (15000, 20000), 
    (20000, 25000), 
    (25000, 30000), 
    (30000, 35000), 
    (35000, 40000), 
    (40000, 45000), 
    (45000, 50000), 
    (50000, 55000), 
    (55000, 60000), 
    (60000, 65000), 
    (65000, 70000), 
    (70000, 75000), 
    (75000, 80000), 
    (80000, 85000), 
    (85000, 90000), 
    (90000, 95000), 
    (95000, 100000), 
    (100000, 125000), 
    (125000, 150000), 
    (150000, 175000), 
    (175000, 200000), 
    (200000, 225000), 
    (225000, 250000), 
    (250000, 275000), 
    (275000, 300000), 
    (300000, 325000), 
    (325000, 350000), 
    (350000, 375000), 
    (375000, 400000), 
    (400000, 425000), 
    (425000, 450000), 
    (450000, 475000), 
    (475000, 500000), 
    (500000, 550000), 
    (550000, 600000), 
    (600000, 650000), 
    (650000, 700000), 
    (700000, 750000), 
    (750000, 800000), 
    (800000, 850000), 
    (850000, 900000), 
    (900000, 950000), 
    (950000, 1000000), 
    (1000000, 1250000), 
    (1250000, 1500000), 
    (1500000, 1750000), 
    (1750000, 2000000), 
    (2000000, 2500000), 
    (2500000, 3000000), 
    (3000000, 4000000), 
    (4000000, 5000000), 
    (5000000, 6000000), 
    (6000000, 7000000), 
    (7000000, 8000000), 
    (8000000, 9000000), 
    (9000000, 10000000), 
    (10000000, 11000000), 
    (11000000, 12000000), 
    (12000000, 13000000), 
    (13000000, 14000000), 
    (14000000, 15000000), 
    (15000000, 16000000), 
    (16000000, 17000000), 
    (17000000, 18000000), 
    (18000000, 19000000), 
    (19000000, 20000000), 
    (20000000, 40000000), 
    (40000000, 60000000), 
    (60000000, 80000000), 
    (80000000, 100000000), 
    (100000000, 200000000), 
    (200000000, 300000000), 
    (300000000, 400000000), 
    (400000000, 500000000), 
    (500000000, 600000000), 
    (600000000, 700000000), 
    (700000000, 800000000), 
    (800000000, 900000000), 
    (900000000, 1000000000)
]

In [8]:
# buckets for variables that are strictly positive values
# buckets' first bin excludes 0 
non_monetary_buckets = monetary_buckets.copy()
non_monetary_buckets[0] = (0, 20)

In [9]:
monetary_bins = pd.IntervalIndex.from_tuples(monetary_buckets)
non_monetary_bins = pd.IntervalIndex.from_tuples(non_monetary_buckets)

In [10]:
monetary_bins_to_enumerated_val = dict(zip(monetary_bins, range(1, len(monetary_bins) + 1)))
non_monetary_bins_to_enumerated_val = dict(zip(non_monetary_bins, range(1, len(non_monetary_bins) + 1)))

In [11]:
enumerated_val_to_range = dict(zip(range(1, len(non_monetary_buckets) + 1), non_monetary_buckets))

## Hot Deck

**2024/01/26 UPDATE**: don't need hot deck imputation as imputation class variables will not be bucketed. Just use KNN imputation.

Questions

- when selecting donors, do we want to reuse donors from the donor pool? _Yes, set max donor use at 5 times._

In [12]:
# need a function to standardize the NAs
# In data, values to impute are denoted by -1

In [13]:
# easy dummy data to work with
iris = load_iris(as_frame=True, return_X_y=True)
iris_df = iris[0]
iris_df["species"] = iris[1]

# deterministically assign some missing values
random.seed(1)
na_mask = random.sample(range(150), 30)

iris_df.iloc[na_mask, 3] = pd.NA
iris_df.iloc[[i + 2 for i in na_mask], 1] = pd.NA

Apply would return (donor_val, idx) for each row that needs to be imputed.

Make a copy of original data and directly modify columns. No inplace changes.

Return index of donor, then increment the counter column to determine how often a row has be used as a donor.

Order of imputation type:

1. Hot deck/1-NN with exact match.
2. 1-NN with nearest neighbour match.
3. Expand donor pool by removing the last listed matching variable.
   - Scenarios where this could happen:
       - All existing donors defined by using full set of matching variables have been used more than `x` times where `x` is the maximum donor uses.
       - Observation to be imputed for has unique matching variable values

With nearest neighbour, one can always be found, it's just a matter of distance. It's difficult to set a blanket constraint for max distance away for a donor. Should we just do the matching variables reduction first?

In [14]:
# read in variable to impute and its imputation class variables as a dictionary?
# ex. 
{
    "income": ["age", "SSIC"],
    "expense": ["age", "size_of_house"]
}

# TODO: 
# add doc string and type hints
# add imputation flag -- done

# iterate through each key-value pair, generating a new column with the imputations?
def hot_deck_by_row(row, pool, var_to_impute, imputation_class_vars, max_donor_use, donor_use_dict, userid_var_name='userid'): # seed=None

    row_post_imputation = row.copy()
    matching_vars_used = matching_vars.copy()
    
    # only impute if var_to_impute is missing for row
    if pd.isna(row[var_to_impute]):

        # exclude donors that have been overused
        overused_donor_indices = [k for k, v in donor_use_dict.items() if v >= max_donor_use]
        pool = pool.drop(overused_donor_indices, axis=0)
        
        # get values that define imputation class
        conds = row[matching_vars_used].to_dict()
        pool = pool[np.apply_along_axis(all, 1, ((pool[conds.keys()] == conds.values()).values))]
    
        # only impute if there are donors
        if len(pool) > 0:
            donor_row = pool.sample(n=1) # with replacement sampling
            donor_val = donor_row[var_to_impute].values.item()
            donor_userid = int(donor_row[userid_var_name].item())

            # impute using donor value
            row_post_imputation[var_to_impute] = donor_val

            # change imputation flag to 1
            imputation_flag_col_name = f'{var_to_impute}_imputation_flag'
            row_post_imputation[imputation_flag_col_name] = 1

            # Update donor userid
            donor_userid_col_name = f'{var_to_impute}_donor_userid'
            row_post_imputation[donor_userid_col_name] = donor_userid
            donor_use_dict[donor_userid] += 1
        
        else:
            # what to do if there are no suitable donors?
            # just pass nothing?
            pass      

    return row_post_imputation

In [15]:
# For imputation class variable order:
# for an automated choice, we can drop variables in the order of the number of levels each variable has
# but SM decisions would yield a better choice

# hot_deck_imputation does imputation for the entire columns

def hot_deck_imputation(data, var_to_impute, matching_vars, max_donor_use=5, userid_var_name='userid'):

    # TODO: check parameters for errors

    data_post_imputation = data.copy()

    # get imputation flag, donor userid and donor count variable names
    imputation_flag_col_name = f'{var_to_impute}_imputation_flag'
    donor_userid_col_name = f'{var_to_impute}_donor_userid'
    donor_count_col_name = f'{var_to_impute}_donor_count'

    # get donor count column name for indexing use later
    count_col_idx = data_post_imputation.columns.get_loc(donor_count_col_name)

    # initialize donor use tracking dictionary
    donor_use_track_dict = {k:0 for k in data_post_imputation[userid_var_name]}

    # Only include donors that don't have missing var_to_impute value
    pool = data_post_imputation[(data_post_imputation[var_to_impute].notna())]
    
    # Do donor imputation
    # NOTE: this doesn't change data in place
    data_post_imputation = data_post_imputation.apply(
        lambda row: hot_deck_by_row(
            row,
            pool,
            var_to_impute,
            matching_vars,
            max_donor_use,
            donor_use_track_dict),
        axis=1)

    # Convert donor_index, imputation flag and count column to integer
    convert_to_int_cols = [donor_userid_col_name, imputation_flag_col_name, donor_count_col_name]
    data_post_imputation[convert_to_int_cols] = data_post_imputation[convert_to_int_cols].astype("int32")

    # Get number of times row is used as donor
    donor_use_by_row = Counter(data_post_imputation[donor_userid_col_name])
    del(donor_use_by_row[-1])
    for k, v in donor_use_by_row.items():
        data_post_imputation.iloc[k, count_col_idx] += v

    return data_post_imputation

In [16]:
# this is called after determining that dropping matching variables is necessary
# ie. exhausted hot deck imputation, KNN imputation (both unconstrained and constrained)

def donor_pool_adj_hot_deck(data, var_to_impute, matching_vars, max_donor_use=5):

    # drop last variable from imputation class variables list
    matching_vars_reduced = matching_vars.copy()
    del matching_vars_reduced[-1]

    # call hot_deck_imputation
    data_post_imputation = hot_deck_imputation(data, var_to_impute, matching_vars_reduced, max_donor_use)

    return data_post_imputation    

In [17]:
iris_imputed_df = iris_df.copy()
iris_imputed_df['userid'] = range(0, len(iris_imputed_df))

matching_vars = ["species", "sepal length (cm)"]
var_to_impute = "sepal width (cm)"
iris_imputed_df = data_preprocessing_store_donor_info(iris_imputed_df, var_to_impute, matching_vars)
iris_imputed_df = hot_deck_imputation(iris_imputed_df, var_to_impute, matching_vars, 1)

In [18]:
iris_imputed_df["sepal width (cm)_donor_count"].value_counts()

sepal width (cm)_donor_count
0    123
1     27
Name: count, dtype: int64

In [19]:
iris_imputed_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species,userid,sepal width (cm)_post_imputation,sepal width (cm)_imputation_flag,sepal width (cm)_imputation_type,sepal width (cm)_imputed_value,sepal width (cm)_donor_userid,sepal width (cm)_donor_count,sepal width (cm)_donor_species,sepal width (cm)_donor_sepal length (cm),sepal width (cm)_donor_distance
0,5.1,3.5,1.4,NaN,0,0,3.5,0,,<NA>,-1,0,<NA>,<NA>,<NA>
1,4.9,3.0,1.4,NaN,0,1,3.0,0,,<NA>,-1,0,<NA>,<NA>,<NA>
2,4.7,3.2,1.3,0.2,0,2,NaN,1,,<NA>,29,0,<NA>,<NA>,<NA>
3,4.6,3.6,1.5,NaN,0,3,NaN,1,,<NA>,22,0,<NA>,<NA>,<NA>
4,5.0,3.6,1.4,0.2,0,4,3.6,0,,<NA>,-1,0,<NA>,<NA>,<NA>


## Nearest Neighbour Imputation

In [20]:
samples = [[0, 0, 2], [1, 0, 0], [0, 0, 1]]
samples2 = [[0, 0, 4], [2, 0, 0], [0, 0, 2]]

In [21]:
samples_trees = []
samples_trees.append(BallTree(samples))
samples_trees.append(BallTree(samples2))
list_samples_dist_ind = []

list_samples_dist_ind.append(
    samples_trees[0].query(
        samples, 
        k=3,
        return_distance=True,
        dualtree=True
    )
)
list_samples_dist_ind.append(
    samples_trees[1].query(
        samples2, 
        k=3,
        return_distance=True,
        dualtree=True
    )
)

In [22]:
[1, 2, 3][2]

3

In [23]:
def nearest_neighbour_imputation(
    data: pd.DataFrame,
    var_to_impute: str,
    imputation_class_vars: List[str],
    missing_values_to_impute: List[int],
    bucket_imputation: bool,
    bins: Optional[pd.IntervalIndex] = None,
    bins_to_labels: Optional[Dict] = None,
    multiple_possible_class_vars: Optional[List[str]] = None,
    max_donor_use: int = 5,
    userid_col_name: str = 'userid'
) -> pd.DataFrame:
    
    """ 1-Nearest Neighbour imputation function.
    
    Parameters
    ----------

    data : pd.Dataframe
        Pandas DataFrame of input data.

    var_to_impute : str
        Name of variable to be imputed.

    imputation_class_vars : list of strings
        Names of variables used to define the imputation class.

    missing_values_to_impute : list of integers
        List of missing values to impute for.

    bucket_imputation : bool
        Boolean indicator for whether to generate an additional, bucketed version of the imputed column.
        
    bins : pd.IntervalIndex, optional
        Only required if bucket_imputation == True.
        Pandas Interval Index type that defines the buckets.
        Note that the left bound is exclusive, right bound is inclusive.
        
    bins_to_labels : dict, optional
        Only required if bucket_imputation == True.
        Dictionary with Pandas Interval Indices as keys and corresponding integer labels as values.

    multiple_possible_class_vars : list of strings, optional
        Names of imputation class variables that can be used in place of the other
        ex. either work income or business earnings can be used as an imputation class variable.

    max_donor_use : int, default = 5
        Maximum number of times a donor can be used.

    userid_col_name : str, default = 'userid'
        Name of user ID variable.

    Returns
    -------

    data_post_imputation : pd.DataFrame
        Pandas DataFrame of output data, with imputations applied to variable to be imputed.
    """
    
    # make a copy to apply imputation
    data_post_imputation = data.copy()
    
    # get imputation details and donor details' variable names
    imputation_flag_col_name = f'{var_to_impute}_imputation_flag'
    imputation_type_col_name = f'{var_to_impute}_imputation_type'
    post_imputation_col_name = f'{var_to_impute}_post_imputation'
    imputed_val_col_name = f'{var_to_impute}_imputed_value'
    donor_userid_col_name = f'{var_to_impute}_donor_userid'
    donor_count_col_name = f'{var_to_impute}_donor_count'
    donor_characteristics_col_names = [f'{var_to_impute}_donor_{var}' for var in imputation_class_vars]
    donor_distance_col_name = f'{var_to_impute}_donor_distance'

    # get column numbers for indexing use later
    data_columns = data_post_imputation.columns
    imputation_flag_col_idx = data_columns.get_loc(imputation_flag_col_name)
    imputation_type_col_idx = data_columns.get_loc(imputation_type_col_name)
    post_imputation_col_idx = data_columns.get_loc(post_imputation_col_name)
    imputed_val_col_idx = data_columns.get_loc(imputed_val_col_name)
    donor_userid_col_idx = data_columns.get_loc(donor_userid_col_name)
    donor_count_col_idx = data_columns.get_loc(donor_count_col_name)
    donor_characteristics_col_idxs = [data_columns.get_loc(var) for var in donor_characteristics_col_names]
    donor_distance_col_idx = data_columns.get_loc(donor_distance_col_name)

    # initialize donor use tracking dictionary
    donor_use_track_dict = {k: 0 for k in data_post_imputation[userid_col_name]}

    # replace post-imputation variable column and imputation class variables with NA
    data_post_imputation[imputation_class_vars] = data_post_imputation[imputation_class_vars].replace(missing_values_to_impute, pd.NA)
    data_post_imputation[post_imputation_col_name] = data_post_imputation[post_imputation_col_name].replace(missing_values_to_impute, pd.NA)
    if multiple_possible_class_vars is not None:
        data_post_imputation[multiple_possible_class_vars] = data_post_imputation[multiple_possible_class_vars].replace(missing_values_to_impute, pd.NA)

    # initialize min-max scaler
    minmax_scaler = MinMaxScaler()
    
    if multiple_possible_class_vars is None:

        # Need to min-max scale imputation class variables before determining nearest neighbours.
        imputation_class_vars_transformed = minmax_scaler.fit_transform(data_post_imputation[imputation_class_vars])
        
        # Use scikit-learn's BallTree function to get nearest neighbours and their indices.
        tree = BallTree(imputation_class_vars_transformed)
        neighbour_distances_indices = tree.query(
            imputation_class_vars_transformed, 
            k=data_post_imputation.shape[0],
            return_distance=True,
            dualtree=True
        )

        for tree_index_num, data_index_num in enumerate(data_post_imputation.index):

            # Search for neighbours if row has missing value
            if pd.isna(data_post_imputation[post_imputation_col_name][data_index_num]):
        
                neighbour_rank = 0
                continue_neighbour_search = True
        
                while continue_neighbour_search:
        
                    donor_idx = neighbour_distances_indices[1][tree_index_num, neighbour_rank]
                    donor_dist = neighbour_distances_indices[0][tree_index_num, neighbour_rank]
                    donor_userid = data_post_imputation[userid_col_name][donor_idx]
        
                    # first check: donor use does not exceed max_donor_use.
                    if donor_use_track_dict[donor_userid] > max_donor_use:
                        neighbour_rank += 1                   
                    
                    # second check: if closest donor is itself, then continue neighbour search
                    elif donor_idx == data_index_num:
                        neighbour_rank += 1
                    
                    # third check: if donor has missing value, then continue neighbour search
                    elif pd.isna(data_post_imputation[post_imputation_col_name][donor_idx]):
                        neighbour_rank += 1
                    
                    # else, donor is eligible; end neighbour search
                    else:
                        continue_neighbour_search = False
        
                        # impute value into post-imputation variable
                        data_post_imputation.iloc[data_index_num, post_imputation_col_idx] = data_post_imputation[post_imputation_col_name][donor_idx]
                    
                        # indicate imputation
                        data_post_imputation.iloc[data_index_num, imputation_flag_col_idx] = 1
    
                        # indicate type of imputation
                        data_post_imputation.iloc[data_index_num, imputation_type_col_idx] = 'KNN'
    
                        # save imputed value
                        data_post_imputation.iloc[data_index_num, imputed_val_col_idx] = data_post_imputation[post_imputation_col_name][donor_idx]
    
                        # save donor's user id
                        data_post_imputation.iloc[data_index_num, donor_userid_col_idx] = donor_userid
    
                        # increment donor's use count in donor use tracking dictionary
                        donor_use_track_dict[donor_userid] += 1
    
                        # save donor characteristics
                        for n, var in enumerate(imputation_class_vars):
                            data_post_imputation.iloc[data_index_num, donor_characteristics_col_idxs[n]] = data_post_imputation[var][donor_idx]
    
                        # save donor distance
                        data_post_imputation.iloc[data_index_num, donor_distance_col_idx] = donor_dist
        
    # if there are imputation class variables that can be used in lieu of the other,
    # then separate distance matrices need to be computed per variable in multiple_possible_class_vars
    else:
        # this list stores tuples of distances and neighbour indices per multiple_possible_class_vars variable
        list_neighbour_distances_indices = []
        
        for var in multiple_possible_class_vars:

            # Need to min-max scale imputation class variables before determining nearest neighbours.
            imputation_class_vars_transformed = minmax_scaler.fit_transform(data_post_imputation[imputation_class_vars + var])
            
             # Use scikit-learn's BallTree function to get nearest neighbours and their indices.
            tree = BallTree(imputation_class_vars_transformed)
            list_neighbour_distances_indices.append(
                tree.query(
                    imputation_class_vars_transformed, 
                    k=data_post_imputation.shape[0],
                    return_distance=True,
                    dualtree=True
                )
            )

        for tree_index_num, data_index_num in enumerate(data_post_imputation.index):

            # Search for neighbours if row has missing value
            if pd.isna(data_post_imputation[post_imputation_col_name][data_index_num]):
        
                neighbour_rank = 0
                continue_neighbour_search = True
        
                while continue_neighbour_search:

                    # Determine which multiple_possible_class_vars variable to use
                    # by finding the first variable with non-missing value
                    pos = 0
                    continue_class_vars_search = True
                    
                    while continue_class_vars_search:
                        # This case only occurs if all variables in multiple_possible_class_vars are missing in a given row.
                        # This should not happen as a base assumption is that imputation class variables have no missingness.
                        if pos >= len(multiple_possible_class_vars):
                            continue_class_vars_search = False
                            userid_missing_class_vars = data_post_imputation[data_index_num][userid_col_name]
                            warnings.warn(
                                f'All variables in multiple_possible_class_vars are missing for user id {userid_missing_class_vars}.'
                            )
                        
                        elif pd.isna(data_post_imputation[multiple_possible_class_vars[pos]]):
                            pos += 1
                        
                        # first variable in multiple_possible_class_vars that is not missing
                        else:
                            continue_class_vars_search = False

                    neighbour_distances_indices = list_neighbour_distances_indices[pos]
        
                    donor_idx = neighbour_distances_indices[1][tree_index_num, neighbour_rank]
                    donor_dist = neighbour_distances_indices[0][tree_index_num, neighbour_rank]
                    donor_userid = data_post_imputation[userid_col_name][donor_idx]
        
                    # first check: donor use does not exceed max_donor_use.
                    if donor_use_track_dict[donor_userid] > max_donor_use:
                        neighbour_rank += 1                   
                    
                    # second check: if closest donor is itself, then continue neighbour search
                    elif donor_idx == data_index_num:
                        neighbour_rank += 1
                    
                    # third check: if donor has missing value, then continue neighbour search
                    elif pd.isna(data_post_imputation[post_imputation_col_name][donor_idx]):
                        neighbour_rank += 1
                    
                    # else, donor is eligible; end neighbour search
                    else:
                        continue_neighbour_search = False
        
                        # impute value into post-imputation variable
                        data_post_imputation.iloc[data_index_num, post_imputation_col_idx] = data_post_imputation[post_imputation_col_name][donor_idx]
                    
                        # indicate imputation
                        data_post_imputation.iloc[data_index_num, imputation_flag_col_idx] = 1
    
                        # indicate type of imputation
                        data_post_imputation.iloc[data_index_num, imputation_type_col_idx] = 'KNN'
    
                        # save imputed value
                        data_post_imputation.iloc[data_index_num, imputed_val_col_idx] = data_post_imputation[post_imputation_col_name][donor_idx]
    
                        # save donor's user id
                        data_post_imputation.iloc[data_index_num, donor_userid_col_idx] = donor_userid
    
                        # increment donor's use count in donor use tracking dictionary
                        donor_use_track_dict[donor_userid] += 1
    
                        # save donor characteristics
                        for n, var in enumerate(imputation_class_vars):
                            data_post_imputation.iloc[data_index_num, donor_characteristics_col_idxs[n]] = data_post_imputation[var][donor_idx]
    
                        # save donor distance
                        data_post_imputation.iloc[data_index_num, donor_distance_col_idx] = donor_dist

    # populate donor use count column
    for row_num, donor_index in enumerate(data_post_imputation[donor_userid_col_name]):
        if donor_index != -1:
            data_post_imputation.iloc[row_num, donor_count_col_idx] = donor_use_track_dict[donor_index]  

    # bucket imputed column if bucket_imputation == True
    if bucket_imputation:
        bucketed_post_imputation_col_name = f'{var_to_impute}_post_imputation_bucketed'
        data_post_imputation[bucketed_post_imputation_col_name] = continuous_to_range(
            data_post_imputation,
            post_imputation_col_name,
            bins,
            bins_to_labels
        )
    
    return data_post_imputation

In [24]:
iris_knn_imputed_df = iris_df.copy()
iris_knn_imputed_df['userid'] = range(0, len(iris_knn_imputed_df))

iris_knn_imputed_df = data_preprocessing_store_donor_info(iris_knn_imputed_df, var_to_impute, matching_vars)

In [25]:
iris_knn_imputed_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species,userid,sepal width (cm)_post_imputation,sepal width (cm)_imputation_flag,sepal width (cm)_imputation_type,sepal width (cm)_imputed_value,sepal width (cm)_donor_userid,sepal width (cm)_donor_count,sepal width (cm)_donor_species,sepal width (cm)_donor_sepal length (cm),sepal width (cm)_donor_distance
0,5.1,3.5,1.4,NaN,0,0,3.5,0,,<NA>,-1,0,<NA>,<NA>,<NA>
1,4.9,3.0,1.4,NaN,0,1,3.0,0,,<NA>,-1,0,<NA>,<NA>,<NA>
2,4.7,NaN,1.3,0.2,0,2,NaN,0,,<NA>,-1,0,<NA>,<NA>,<NA>
3,4.6,NaN,1.5,NaN,0,3,NaN,0,,<NA>,-1,0,<NA>,<NA>,<NA>
4,5.0,3.6,1.4,0.2,0,4,3.6,0,,<NA>,-1,0,<NA>,<NA>,<NA>


In [26]:
# apply nearest neighbour imputation
iris_knn_imputed_df = nearest_neighbour_imputation(
    iris_knn_imputed_df,
    var_to_impute,
    matching_vars,
    [-1, 88],
    True,
    non_monetary_bins,
    non_monetary_bins_to_enumerated_val
)

In [27]:
iris_knn_imputed_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species,userid,sepal width (cm)_post_imputation,sepal width (cm)_imputation_flag,sepal width (cm)_imputation_type,sepal width (cm)_imputed_value,sepal width (cm)_donor_userid,sepal width (cm)_donor_count,sepal width (cm)_donor_species,sepal width (cm)_donor_sepal length (cm),sepal width (cm)_donor_distance,sepal width (cm)_post_imputation_bucketed
0,5.1,3.5,1.4,NaN,0,0,3.5,0,,<NA>,-1,0,<NA>,<NA>,<NA>,1
1,4.9,3.0,1.4,NaN,0,1,3.0,0,,<NA>,-1,0,<NA>,<NA>,<NA>,1
2,4.7,NaN,1.3,0.2,0,2,3.2,1,KNN,3.2,29,1,0,4.7,0.0,1
3,4.6,NaN,1.5,NaN,0,3,3.2,1,KNN,3.2,47,1,0,4.6,0.0,1
4,5.0,3.6,1.4,0.2,0,4,3.6,0,,<NA>,-1,0,<NA>,<NA>,<NA>,1
5,5.4,NaN,1.7,NaN,0,5,3.9,1,KNN,3.9,16,1,0,5.4,0.0,1
6,4.6,3.4,1.4,0.3,0,6,3.4,0,,<NA>,-1,0,<NA>,<NA>,<NA>,1
7,5.0,NaN,1.5,NaN,0,7,3.3,1,KNN,3.3,49,2,0,5.0,0.0,1
8,4.4,2.9,1.4,0.2,0,8,2.9,0,,<NA>,-1,0,<NA>,<NA>,<NA>,1
9,4.9,NaN,1.5,0.1,0,9,3.6,1,KNN,3.6,37,1,0,4.9,0.0,1


In [28]:
iris_knn_imputed_df[iris_knn_imputed_df['sepal width (cm)_donor_count'] == 2]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species,userid,sepal width (cm)_post_imputation,sepal width (cm)_imputation_flag,sepal width (cm)_imputation_type,sepal width (cm)_imputed_value,sepal width (cm)_donor_userid,sepal width (cm)_donor_count,sepal width (cm)_donor_species,sepal width (cm)_donor_sepal length (cm),sepal width (cm)_donor_distance,sepal width (cm)_post_imputation_bucketed
7,5.0,NaN,1.5,NaN,0,7,3.3,1,KNN,3.3,49,2,0,5.0,0.0,1
26,5.0,NaN,1.6,NaN,0,26,3.3,1,KNN,3.3,49,2,0,5.0,0.0,1
28,5.2,NaN,1.4,0.2,0,28,3.5,1,KNN,3.5,27,2,0,5.2,0.0,1
32,5.2,NaN,1.5,0.1,0,32,3.5,1,KNN,3.5,27,2,0,5.2,0.0,1
101,5.8,NaN,5.1,1.9,2,101,2.7,1,KNN,2.7,142,2,2,5.8,0.0,1
114,5.8,NaN,5.1,NaN,2,114,2.7,1,KNN,2.7,142,2,2,5.8,0.0,1
116,6.5,NaN,5.5,1.8,2,116,3.2,1,KNN,3.2,110,2,2,6.5,0.0,1
117,7.7,NaN,6.7,2.2,2,117,2.6,1,KNN,2.6,118,2,2,7.7,0.0,1
122,7.7,NaN,6.7,2.0,2,122,2.6,1,KNN,2.6,118,2,2,7.7,0.0,1
147,6.5,NaN,5.2,2.0,2,147,3.2,1,KNN,3.2,110,2,2,6.5,0.0,1


In [29]:
iris_knn_imputed_df.apply(pd.isna).apply(sum)

sepal length (cm)                              0
sepal width (cm)                              30
petal length (cm)                              0
petal width (cm)                              30
species                                        0
userid                                         0
sepal width (cm)_post_imputation               0
sepal width (cm)_imputation_flag               0
sepal width (cm)_imputation_type               0
sepal width (cm)_imputed_value               120
sepal width (cm)_donor_userid                  0
sepal width (cm)_donor_count                   0
sepal width (cm)_donor_species               120
sepal width (cm)_donor_sepal length (cm)     120
sepal width (cm)_donor_distance              120
sepal width (cm)_post_imputation_bucketed      0
dtype: int64

### BallTree

Determine nearest neighbours using `BallTree`, then query to get the nearest neighbours of data points that require imputation.

Advantages:

- able to just construct the k-nearest neighbours "table", lending to more flexibility. We can get more neighbours if required!
- can get indices of donors

Disadvantages:

- not as optimized as the `KNNImputer` function. However, this approach retrieve the donors' index, which is critical in determining if a donor has reached its max use.
- will have to filter nearest neighbours to exclude neighbours with missing values. 

### NearestNeighbors

Determine nearest neighbours using `NearestNeighbors`.

Advantages:

- can get indices of donors

Disadvantages:

- if `NearestNeighbors` is only called once, then missing values cannot be excluded. However, donors could possibly have missing values themselves.
- alternatively, call `NearestNeighbors` each time you need to impute. `NearestNeighbors` should be fit on the subset of dataframe containing all possible valid donors (ie. does not have missing value for variable to be imputed and does not exceed donor use count) and the row that requires imputing. Unlike using `BallTree`, we do not have to go through the k-nearest neighbours to discard neighbours that have missing values. However, `BallTree` would only have to be run once whereas `NearestNeighbors` would have to be run multiple times.
- sometimes, the observation itself is its own nearest neighbour. Problem is that this behaviour is not consistent. Would have to keep the top 2 neighbours to determine if the nearest point is itself?

# Ratio/Trend Imputation

See "Historical Imputation in RHS_forStatCan.docx" for ratio/trend imputation specification. Only work income (including bonus and net earnings/profits) and housing loan value will be subjected to this imputation approach.

To facilitate testing, a subset of the data given by the clients will be used.

Questions

1. What does the attrition rate look like for survey participants?
2. Can new participants join midway through the study ex. participant joins in W5?
3. What should be done if the imputation class variable is missing ex. number of jobs is missing for a record that requires imputation. Assumption is that we fall back to donor imputation.
4. Specific to housing loan value ratio imputation:

   - How do we identify if respondent reported same housing in adjacent waves? Is there an indicator for whether a person changed housing, a location field to track etc.?
   - Should we condition that the number of houses reported in previous wave should be equal to the number of houses report in current wave in order to do ratio imputation? The current condition is that the number of houses in previous wave $\ge$ number of houses in current wave.
5. Specific to income/bonus/net earnings/profit:

    - How do we identify if respondent changed jobs across waves? Are there job title and employer name fields to track or do we check just using SSIC/SSOC?
    - For the post-imputation reasonableness check, should it also apply to net earnings/profit? Currently, only a reasonableness check is done for work income. _Business profits may fluctuate more compared to work income, hence not having a ceiling for imputed values._
6. What values do we want to impute for? Ex. -1 only, (-1, 88)?

ASSUMPTIONS

- want to impute -1 and 88

In [30]:
# set folder and file names
data_folder = r'\\fld6filer\icmic-cciim\CONSULTATION\IMPUDON\python-redesign\data'
w3_w4_subset_file = 'W3_W4_combined_for_trend_imputation.xlsx'

# w4_data_file = 'W4_Data_StatCan_23082023_1.xlsx'
# w4_data_path = Path(data_folder) / w4_data_file
# w3_data_file = 'W3_Data_StatCan_25082023.xlsx'
# w3_data_path = Path(data_folder) / w3_data_file

# # path for subset of W4 data to work with
# w4_subset_for_trend_imputation_file = 'W4_subset_for_trend_imputation.xlsx'
# w4_subset_for_trend_imputation_path = Path(data_folder) / w4_subset_for_trend_imputation_file

# # read in Wave 3 data
# w3_data = pd.read_excel(w3_data_path)

# # read in Wave 4 data
# w4_data = pd.read_excel(w4_data_path)

# # save subset of W4 data to work with
# w4_data[w4_data.columns[w4_data.columns.isin(w3_data.columns)]].to_excel(w4_subset_for_trend_imputation_path, index=False)
# w4_subset_data = pd.read_excel(w4_subset_for_trend_imputation_path)

In [31]:
w3_data.head()

NameError: name 'w3_data' is not defined

In [ ]:
w4_subset_data.head()

In [ ]:
w3_data.columns[w3_data.columns.str.endswith('_c')]

Combine W4 and W3 data to do trend/ratio imputation.

In [ ]:
tst = w4_subset_data.join(w3_data.set_index('userid'), on='userid', how='left', rsuffix='_prev_wave')

In [ ]:
tst.to_excel(Path(data_folder) / 'W3_W4_combined_for_trend_imputation.xlsx', index=False)

## Utility Functions for Historical Imputation

In [ ]:
# have a general trend imputation function 
def ratio_imputation(avg_current_wave: float, avg_previous_wave: float, val_past_wave: float) -> float:
    
    """Function for applying ratio imputation.

    Parameters
    ----------

    avg_current_wave : float
        Average value in the current wave.

    avg_previous_wave : float
        Average value in the previous wave.

    val_past_wave : float
        Value in the previous wave.

    Returns
    -------

    (avg_current_wave / avg_previous_wave) * val_past_wave : float
        Imputed value using ratio imputation.

    """
    return (avg_current_wave / avg_previous_wave) * val_past_wave

In [ ]:
# have function that get imputation groups for historical imputation
# and pre-calculates average value, number of records per group
# for example,
house_loan_donor_pool_one_house = tst[
    (~tst.Value_Hse_Loan_prev_wave.isin(values_to_impute))
    & (~tst.Value_Hse_Loan.isin(values_to_impute))
    & (tst.Total_Prop == 1)
    & (tst.Total_Prop_prev_wave == 1)
].groupby(['Total_Prop_prev_wave', 'Whether_HDB_Housing_prev_wave'])[['Value_Hse_Loan', 'Value_Hse_Loan_prev_wave']].agg(['count', 'mean']).reset_index()

house_loan_donor_pool_one_house.columns = ['_'.join(col).rstrip('_') for col in house_loan_donor_pool_one_house.columns]

# def get_imputation_group_details(
#     data: pd.DataFrame, 
#     values_to_impute: list[str],
#     filter_conds
# ):
#     pass

In [ ]:
mask = (~tst.Value_Hse_Loan_prev_wave.isin(values_to_impute)) & (~tst.Value_Hse_Loan.isin(values_to_impute)) & (tst.Total_Prop == 1) & (tst.Total_Prop_prev_wave == 1)

In [ ]:
tst[mask]

## Housing Loan Imputation

First, impute for housing loan value.

Segment the previous wave's data by housing type outside of the row-by-row imputation step.

Variables that end with the suffix `_c` denote imputed values.

In [ ]:
house_loan_donor_pool_one_house = tst[
    (~tst.Value_Hse_Loan_prev_wave.isin(values_to_impute))
    & (~tst.Value_Hse_Loan.isin(values_to_impute))
    & (tst.Total_Prop == 1)
    & (tst.Total_Prop_prev_wave == 1)
].groupby(['Total_Prop_prev_wave', 'Whether_HDB_Housing_prev_wave'])[['Value_Hse_Loan', 'Value_Hse_Loan_prev_wave']].agg(['count', 'mean']).reset_index()

house_loan_donor_pool_one_house.columns = ['_'.join(col).rstrip('_') for col in house_loan_donor_pool_one_house.columns]

In [ ]:
house_loan_donor_pool_one_house

In [ ]:
sample_row = tst[tst.userid == 35720].copy()
sample_row[['Value_Hse_Loan']] = pd.NA

In [ ]:
house_loan_donor_pool_multiple = tst[
    (~tst.Value_Hse_Loan_prev_wave.isin(values_to_impute))
    & (~tst.Value_Hse_Loan.isin(values_to_impute))
    & (~tst.Total_Prop.isin(values_to_impute))
    & (tst.Total_Prop > 1)
    & (tst.Total_Prop_prev_wave >= tst.Total_Prop)
][['Value_Hse_Loan', 'Value_Hse_Loan_prev_wave']]
#.groupby(['Total_Prop_prev_wave', 'Total_Prop'])[['Value_Hse_Loan', 'Value_Hse_Loan_prev_wave']].agg(['count', 'mean']).reset_index()

In [ ]:
house_loan_donor_pool_multiple.apply(np.mean)

Questions

- should I just 'hardcode' the variable names, or should they be function parameters? Need to ask clients if variable names will stay consistent

In [ ]:
house_loan_donor_pool_multiple

In [ ]:
values_to_impute = [-1, 88]

In [ ]:
# do on a row basis first, then apply to entire data set
# probably take in variable-dependent steps/function as parameters:
# functions for condition checking -- income_cond_check, house_loan_cond_check
# grouped dataframes for generating the scaling ratio for imputation? 

def housing_historical_imputation_by_row(
    row: pd.DataFrame, 
    var_current_wave: str, 
    var_previous_wave: str,
    var_number_properties: str,
    var_hdb_indicator_current_wave: str, 
    var_hdb_indicator_previous_wave: str,
    imputation_group_one_house: pd.DataFrame, 
    imputation_group_multi_house: pd.DataFrame,
    missing_values_to_impute: list[int],
    min_records_req: int
) -> pd.DataFrame:

    """Historical imputation function for housing loan value, applied to a single row.

    Row-wise function that will be applied to the entire column in the 
    historical_imputation_housing function.
    
    Parameters
    ----------

    row : pd.Dataframe
        A single row of a Pandas DataFrame.

    var_current_wave : str
        Name of variable in current wave to be imputed.

    var_previous_wave : str
        Name of variable in previous wave to be used for ratio imputation.

    var_number_properties : str
        Name of number of properties variable in current wave.

    var_hdb_indicator_current_wave : str
        Name of HDB housing indicator variable in current wave.
        
    var_hdb_indicator_previous_wave : str
        Name of HDB housing indicator variable in previous wave.
    
    imputation_group_one_house : pd.DataFrame
        Pandas DataFrame containing the imputation groups for respondents with 1 house.
        Columns of this DataFrame are: HDB housing indicator, which defines the 
        imputation classes, count of observations per imputation class, and 
        mean housing loan values in the current and previous waves.

    imputation_group_multi_house : pd.DataFrame
        Pandas DataFrame containing the imputation groups for respondents with multiple houses.
        Columns of this DataFrame are: mean housing loan values in the current and previous waves.

    missing_values_to_impute : list[int]
        List of missing values to impute for.

    min_records_req : int
        Minimum number of non-missing housing loan values for previous and current waves in 
        imputation class.

    Returns
    -------

    row_post_imputation : pd.DataFrame
        Pandas DataFrame row of output data, with imputation applied to a row of the
        housing loan value variable.
    """

    # make copy of row to apply imputation
    row_post_imputation = row.copy()

    # get name of post imputation column
    post_imputation_col_name = f'{var_current_wave}_post_imputation'
    imputed_val_col_name = f'{var_current_wave}_imputed_value'

    # get names of counts and mean / average columns
    record_count_current_wave = f'{var_current_wave}_count'
    record_count_previous_wave = f'{var_previous_wave}_count'
    avg_current_wave = f'{var_current_wave}_mean'
    avg_previous_wave = f'{var_previous_wave}_mean'
    
    # only impute if:
    # var_current_wave is missing for row
    # var_previous_wave was not imputed
    # same housing for previous and current waves
    if pd.isna(row_post_imputation[post_imputation_col_name]) & (~pd.isna(row_post_imputation[var_previous_wave])):
        
        # if there is only 1 property
        if row_post_imputation[var_number_properties] == 1:

            # only impute if there are sufficient records based on imputation parameters
            if (imputation_group_one_house[
                imputation_group_one_house[var_hdb_indicator_previous_wave] == row_post_imputation[var_hdb_indicator_current_wave]
            ][[record_count_current_wave, record_count_previous_wave]].values > min_records_req).all():

                # average housing loan of current wave
                avg_val_current_wave = imputation_group_one_house[
                    imputation_group_one_house[var_hdb_indicator_previous_wave] == row_post_imputation[var_hdb_indicator_current_wave]
                ][avg_current_wave].values[0]

                # average housing loan of previous wave
                avg_val_previous_wave = imputation_group_one_house[
                    imputation_group_one_house[var_hdb_indicator_previous_wave] == row_post_imputation[var_hdb_indicator_current_wave]
                ][avg_previous_wave].values[0]

                # reported value in previous wave
                reported_val_previous_wave = row_post_imputation[var_previous_wave]

                # impute using historical ratio imputation
                imputed_val = ratio_imputation(avg_val_current_wave, avg_val_previous_wave, reported_val_previous_wave)
                row_post_imputation[post_imputation_col_name] = imputed_val
                row_post_imputation[imputed_val_col_name] = imputed_val
                
                # change imputation flag to 1
                imputation_flag_col_name = f'{var_current_wave}_imputation_flag'
                row_post_imputation[imputation_flag_col_name] = 1

                # set imputation type to Historical
                imputation_type_col_name = f'{var_current_wave}_imputation_type'
                row_post_imputation[imputation_type_col_name] = 'Historical'

        # if there are multiple properties
        elif row_post_imputation[var_number_properties] > 1:

            # only impute if there are sufficient records based on imputation parameters
            if imputation_group_multi_house.shape[0] > min_records_req:

                # average housing loan of current wave
                avg_val_current_wave = np.mean(imputation_group_multi_house[var_current_wave])

                # average housing loan of previous wave
                avg_val_previous_wave = np.mean(imputation_group_multi_house[var_previous_wave])

                # reported value in previous wave
                reported_val_previous_wave = row_post_imputation[var_previous_wave]

                # impute using historical ratio imputation
                imputed_val = ratio_imputation(avg_val_current_wave, avg_val_previous_wave, reported_val_previous_wave)
                row_post_imputation[post_imputation_col_name] = imputed_val
                row_post_imputation[imputed_val_col_name] = imputed_val
    
                # change imputation flag to 1
                imputation_flag_col_name = f'{var_current_wave}_imputation_flag'
                row_post_imputation[imputation_flag_col_name] = 1

                # set imputation type to Historical
                imputation_type_col_name = f'{var_current_wave}_imputation_type'
                row_post_imputation[imputation_type_col_name] = 'Historical'

    return row_post_imputation

In [ ]:
def historical_imputation_housing(
    data: pd.DataFrame, 
    var_current_wave: str, 
    var_previous_wave: str,
    var_number_properties: str,
    var_hdb_indicator_current_wave: str, 
    var_hdb_indicator_previous_wave: str,
    imputation_group_one_house: pd.DataFrame, 
    imputation_group_multi_house: pd.DataFrame,
    missing_values_to_impute: list[int],
    min_records_req: int = 30
) -> pd.DataFrame:

    """Historical imputation function for housing loan value for entire data set.

    Wrapper function to housing_historical_imputation_by_row function. 
    Applies historical imputation to entire housing loan value column.
    
    Parameters
    ----------

    data : pd.Dataframe
        Pandas DataFrame of input data.

    var_current_wave : str
        Name of variable in current wave to be imputed.

    var_previous_wave : str
        Name of variable in previous wave to be used for ratio imputation.

    var_number_properties : str
        Name of number of properties variable in current wave.

    var_hdb_indicator_current_wave : str
        Name of HDB housing indicator variable in current wave.
        
    var_hdb_indicator_previous_wave : str
        Name of HDB housing indicator variable in previous wave.

    imputation_group_one_house : pd.DataFrame
        Pandas DataFrame containing the imputation groups for respondents with 1 house.
        Columns of this DataFrame are: HDB housing indicator, which defines the 
        imputation classes, count of observations per imputation class, and 
        mean housing loan values in the current and previous waves.

    imputation_group_multi_house : pd.DataFrame
        Pandas DataFrame containing the imputation groups for respondents with multiple houses.
        Columns of this DataFrame are: mean housing loan values in the current and previous waves.

    missing_values_to_impute : list[int]
        List of missing values to impute for.

    min_records_req : int (default = 30)
        Minimum number of non-missing housing loan values for previous and current waves in 
        imputation class.

    Returns
    -------

    data_post_imputation : pd.DataFrame
        Pandas DataFrame of output data, with imputations applied to housing loan value.
    """

    # TODO: check parameters for errors

    # make copy of data to do imputations on
    data_post_imputation = data.copy()

    # get name of post imputation column
    post_imputation_col_name = f'{var_current_wave}_post_imputation'

    # replace missing values in post-imputation variable and its previous wave's variable columns with NA
    data_post_imputation[post_imputation_col_name] = data_post_imputation[post_imputation_col_name].replace(missing_values_to_impute, pd.NA)
    data_post_imputation[var_previous_wave] = data_post_imputation[var_previous_wave].replace(missing_values_to_impute, pd.NA)
    
    # Do historical imputation for housing variables
    data_post_imputation = data_post_imputation.apply(
        lambda row: housing_historical_imputation_by_row(
            row, 
            var_current_wave, 
            var_previous_wave, 
            var_number_properties,
            var_hdb_indicator_current_wave, 
            var_hdb_indicator_previous_wave,
            imputation_group_one_house, 
            imputation_group_multi_house,
            missing_values_to_impute,
            min_records_req
        ),
        axis=1)

    return data_post_imputation

In [ ]:
# tst_post_imputation_housing = tst.replace(values_to_impute, pd.NA)
tst_post_imputation_housing = data_preprocessing_store_donor_info(tst, 'Value_Hse_Loan', ['Total_Purchase_Price', 'Q140a_1_1'])
tst_post_imputation_housing = historical_imputation_housing(
    tst_post_imputation_housing,
    'Value_Hse_Loan',
    'Value_Hse_Loan_prev_wave',
    'Total_Prop',
    'Whether_HDB_Housing',
    'Whether_HDB_Housing_prev_wave',
    house_loan_donor_pool_one_house,
    house_loan_donor_pool_multiple,
    [-1, 88]
)                                                    

In [ ]:
tst_post_imputation_housing[tst_post_imputation_housing.Value_Hse_Loan_imputation_flag == 1][
[
    'Total_Purchase_Price',
    'Total_Purchase_Price_prev_wave',
    'Whether_HDB_Housing',
    'Whether_HDB_Housing_prev_wave',
    'Total_Prop',
    'Total_Prop_prev_wave',
    'Value_Hse_Loan',
    'Value_Hse_Loan_prev_wave',
    'Value_Hse_Loan_post_imputation',
    'Value_Hse_Loan_imputation_flag',
    'Value_Hse_Loan_imputation_type',
    'Value_Hse_Loan_imputed_value'
    ]
]

In [ ]:
tst_post_imputation_housing[tst_post_imputation_housing.Value_Hse_Loan_imputation_flag == 1].Total_Prop.value_counts()

In [ ]:
tst.Value_Hse_Loan.isin(values_to_impute).sum()

In [ ]:
tst_post_imputation_housing.Value_Hse_Loan_post_imputation.isin(values_to_impute).sum()

## Work income, bonuses, net earnings imputation

QUESTION: is historical imputation only applicable for unbucketed values? _Yes._

- Q128_1: total number of jobs or businesses held
- GO_Q130a_1: SSIC of main job/business
- Q140a_1_1: latest gross monthly income from main job
- Q142a_1_1: bonus received in last 12 months for main job
- Q143a_1_1: net earnings, profits or income from assisting family business for main business

In [ ]:
tst[['Q140a_1_1', 'Q142a_1_1', 'Q143a_1_1']].apply(lambda x: x.isna().sum())

In [ ]:
tst[(tst.Q140a_1_1 == -1) & (~tst.Q140a_1_1_prev_wave.isna()) & (tst.Q140a_1_1_prev_wave != -1)]

In [ ]:
sample_row_work_income = tst[tst.userid == 37309].copy()
# sample_row_work_income['Q140a_1_1'] = pd.NA
sample_row_work_income['Q140a_1_1_imputation_flag'] = 0

In [ ]:
work_income_imputation_group = tst[
    (~tst.Q140a_1_1_prev_wave.isin(values_to_impute)) # no missing values in current wave
    & (~tst.Q140a_1_1.isin(values_to_impute)) # no missing values in previous wave
    & (tst.Q128_1 == tst.Q128_1_prev_wave) # number of jobs equal in previous and current waves
].groupby(['GO_Q130a_1'])[['Q140a_1_1', 'Q140a_1_1_prev_wave']].agg(['count', 'mean']).reset_index()

work_income_imputation_group.columns = ['_'.join(col).rstrip('_') for col in work_income_imputation_group.columns]

In [ ]:
work_income_imputation_group.head()

In [ ]:
bonus_imputation_group = tst[
    (~tst.Q142a_1_1_prev_wave.isin(values_to_impute)) # no missing values in current wave
    & (~tst.Q142a_1_1.isin(values_to_impute)) # no missing values in previous wave
    & (tst.Q128_1 == tst.Q128_1_prev_wave) # number of jobs equal in previous and current waves
].groupby(['GO_Q130a_1'])[['Q142a_1_1', 'Q142a_1_1_prev_wave']].agg(['count', 'mean']).reset_index()

bonus_imputation_group.columns = ['_'.join(col).rstrip('_') for col in bonus_imputation_group.columns]

In [ ]:
bonus_imputation_group = tst[
    (~tst.Q142a_1_1_prev_wave.isin(values_to_impute)) # no missing values in current wave
    & (~tst.Q142a_1_1.isin(values_to_impute)) # no missing values in previous wave
    & (tst.Q128_1 == tst.Q128_1_prev_wave) # number of jobs equal in previous and current waves
].groupby(['GO_Q130a_1'])[['Q142a_1_1', 'Q142a_1_1_prev_wave']].agg(['count', 'mean']).reset_index()

bonus_imputation_group.columns = ['_'.join(col).rstrip('_') for col in bonus_imputation_group.columns]

In [ ]:
def income_historical_imputation_by_row(
    row: pd.DataFrame, 
    var_current_wave: str, 
    var_previous_wave: str,
    num_jobs_current_wave: str,
    num_jobs_previous_wave: str,
    ssic: str,
    imputation_group: pd.DataFrame,
    missing_values_to_impute: list[int],
    min_records_req: int
) -> pd.DataFrame:

    """Historical imputation function for earnings-related variables, applied to a single row.

    Row-wise function that will be applied to the entire column in the 
    historical_imputation_income function.
    
    Parameters
    ----------

    row : pd.Dataframe
        A single row of a Pandas DataFrame.

    var_current_wave : str
        Name of variable in current wave to be imputed.

    var_previous_wave : str
        Name of variable in previous wave to be used for ratio imputation.

    num_jobs_current_wave : str
        Number of jobs / businesses held by respondent in the current wave.

    num_jobs_previous_wave : str
        Number of jobs / businesses held by respondent in the previous wave.

    ssic : str
        SSIC of respondent in current wave.

    imputation_group : pd.DataFrame
        Pandas DataFrame containing the imputation groups, which are defined by
        the job's / business' Singapore Standard Industrial Classification (SSIC).
        Columns of this DataFrame are: SSIC, which defines the imputation classes, 
        count of observations per imputation class, and mean earnings values 
        in the current and previous waves.

    missing_values_to_impute : list[int]
        List of missing values to impute for.

    min_records_req : int
        Minimum number of non-missing earnings value for previous and current waves in 
        imputation class.

    Returns
    -------

    row_post_imputation : pd.DataFrame
        data_post_imputation : pd.DataFrame
        Pandas DataFrame of output data, with imputations applied to housing loan value.
    """

    # make copy of row to apply imputation
    row_post_imputation = row.copy()

    # get names of post imputation columns
    post_imputation_col_name = f'{var_current_wave}_post_imputation'
    imputed_val_col_name = f'{var_current_wave}_imputed_value'

    # get names of counts and mean / average columns
    record_count_current_wave = f'{var_current_wave}_count'
    record_count_previous_wave = f'{var_previous_wave}_count'
    avg_current_wave = f'{var_current_wave}_mean'
    avg_previous_wave = f'{var_previous_wave}_mean'
    
    # only impute if:
    # var_current_wave is missing for row
    # var_previous_wave was not imputed
    # same number of jobs in previous and current waves
    if (pd.isna(row_post_imputation[post_imputation_col_name]) & 
        (~pd.isna(row_post_imputation[var_previous_wave])) & 
        (row_post_imputation[num_jobs_current_wave] == row_post_imputation[num_jobs_previous_wave])):

        # only impute if there are sufficient records based on imputation parameters
        if (imputation_group[imputation_group[ssic] == row_post_imputation[ssic]
            ][[record_count_current_wave, record_count_previous_wave]].values > min_records_req).all():

            # average value of current wave's imputation group
            avg_val_current_wave = imputation_group[imputation_group[ssic] == row_post_imputation[ssic]][avg_current_wave]

            # average value of previous wave's imputation group
            avg_val_previous_wave = imputation_group[imputation_group[ssic] == row_post_imputation[ssic]][avg_previous_wave]

            # reported value in previous wave
            reported_val_previous_wave = row_post_imputation[var_previous_wave]

            # impute using historical ratio imputation
            imputed_val = ratio_imputation(avg_val_current_wave, avg_val_previous_wave, reported_val_previous_wave).values[0]
            row_post_imputation[post_imputation_col_name] = imputed_val
            row_post_imputation[imputed_val_col_name] = imputed_val

            # change imputation flag to 1
            # check with clients: do we want to name flag variable as f'{var_current_wave}_c'?
            imputation_flag_col_name = f'{var_current_wave}_imputation_flag'
            row_post_imputation[imputation_flag_col_name] = 1

            # set imputation type to Historical
            imputation_type_col_name = f'{var_current_wave}_imputation_type'
            row_post_imputation[imputation_type_col_name] = 'Historical'

    return row_post_imputation

In [ ]:
def historical_imputation_income(
    data: pd.DataFrame, 
    var_current_wave: str, 
    var_previous_wave: str,
    num_jobs_current_wave: str,
    num_jobs_previous_wave: str,
    ssic: str,
    imputation_group: pd.DataFrame, 
    missing_values_to_impute: list[int],
    min_records_req: int = 30
):
    
    """Historical imputation function for earnings-related variable for entire data set.

    Wrapper function to income_historical_imputation_by_row function. 
    Applies historical imputation to entire earnings-related variable column.
    
    Parameters
    ----------

    data : pd.Dataframe
        Pandas DataFrame of input data.

    var_current_wave : str
        Name of variable in current wave to be imputed.

    var_previous_wave : str
        Name of variable in previous wave to be used for ratio imputation.

    num_jobs_current_wave : str
        Number of jobs / businesses held by respondent in the current wave.

    num_jobs_previous_wave : str
        Number of jobs / businesses held by respondent in the previous wave.

    ssic : str
        SSIC of respondent in current wave.

    imputation_group : pd.DataFrame
        Pandas DataFrame containing the imputation groups, which are defined by
        the job's / business' Singapore Standard Industrial Classification (SSIC).
        Columns of this DataFrame are: SSIC, which defines the imputation classes, 
        count of observations per imputation class, and mean earnings values 
        in the current and previous waves.

    missing_values_to_impute : list[int]
        List of missing values to impute for.

    min_records_req : int (default = 30)
        Minimum number of non-missing earnings values for previous and current waves in 
        imputation class.

    Returns
    -------

    data_post_imputation : pd.DataFrame
        Pandas DataFrame of output data, with imputations applied to earnings-related variable.
    """

    # TODO: check parameters for errors outside of function

    # make copy of data to do imputations on
    data_post_imputation = data.copy()

    # get name of post imputation column
    post_imputation_col_name = f'{var_current_wave}_post_imputation'

    # replace missing values in post-imputation variable and its previous wave's variable columns with NA
    data_post_imputation[post_imputation_col_name] = data_post_imputation[post_imputation_col_name].replace(missing_values_to_impute, pd.NA)
    data_post_imputation[var_previous_wave] = data_post_imputation[var_previous_wave].replace(missing_values_to_impute, pd.NA)
    
    # Do historical imputation for income variables
    data_post_imputation = data_post_imputation.apply(
        lambda row: income_historical_imputation_by_row(
            row,
            var_current_wave, 
            var_previous_wave,
            num_jobs_current_wave,
            num_jobs_previous_wave,
            ssic,
            imputation_group, 
            missing_values_to_impute,
            min_records_req
        ),
        axis=1)

    return data_post_imputation

In [ ]:
tst.Q140a_1_1.isna().value_counts()

In [ ]:
work_income_imputation_group

In [ ]:
# tst_post_imputation_housing = tst.replace(values_to_impute, pd.NA)
tst_post_imputation_work_income = data_preprocessing_store_donor_info(tst, 'Q140a_1_1', ['GO_Q130a_1', 'GO_Q131a_1'])
tst_post_imputation_work_income = historical_imputation_income(
    tst_post_imputation_work_income,
    'Q140a_1_1',
    'Q140a_1_1_prev_wave',
    'Q128_1',
    'Q128_1_prev_wave',
    'GO_Q130a_1',
    work_income_imputation_group,
    [-1, 88, pd.NA]
)                       

In [ ]:
tst_post_imputation_work_income.Q140a_1_1_imputation_flag.value_counts()

In [ ]:
tst_post_imputation_work_income[tst_post_imputation_work_income.Q140a_1_1_imputation_flag == 1][
    [
        'Q128_1',
        'Q128_1_prev_wave',
        'GO_Q130a_1',
        'GO_Q130a_1_prev_wave',
        'Q140a_1_1',
        'Q140a_1_1_prev_wave',
        'Q140a_1_1_post_imputation',
        'Q140a_1_1_imputed_value',
        'Q140a_1_1_imputation_flag',
        'Q140a_1_1_imputation_type'
    ]
]